# Aquifer data

## Imports

In [ ]:
#pip install geopy pandas sqlite3 folium

In [ ]:
# Datasets
import pandas as pd
import sqlite3

# Geographical data plotting
import folium
from folium.map import Popup
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
from geopy.distance import distance

# Time
import time

## Loading the database

In [ ]:
# Connect to the SQLite database
conn = sqlite3.connect('../data/external/data.db')

# Get a cursor object
cursor = conn.cursor()

# List all of the tables
tables = cursor.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
table_names = [table[0] for table in tables]
print("Tables in the database:", table_names)

# Load data from each table into a pandas DataFrame
dataframes = {table: pd.read_sql_query(f"SELECT * FROM {table}", conn) for table in table_names}

# Name the datasets that we are going to work with
aquifer_stations = dataframes[table_names[4]]
aquifer_measurements = dataframes[table_names[5]]
weather_locations = dataframes[table_names[6]]
weather = dataframes[table_names[7]]

print("aquifer stations:")
print(aquifer_stations.head())
print("aquifer measurements:")
print(aquifer_measurements.head())
print("weather stations:")
print(weather_locations.head())
print("Weather measurements:")
print(weather.head())


## Data manipulation

### Aquifer data

In [ ]:
# Convert 'date' column to datetime
aquifer_measurements['date'] = pd.to_datetime(aquifer_measurements['date'])

# Filter data for years 2010 to 2017
aquifer_measurements = aquifer_measurements[aquifer_measurements['date'].dt.year.between(2010, 2017)]

# Group by station_id and count the number of unique years
station_years_counts = aquifer_measurements.groupby('station_id')['date'].apply(lambda x: x.dt.year.nunique())

# Filter station_ids that have instances in all years from 2010 to 2017
station_ids_all_years = station_years_counts[station_years_counts == 8].index.tolist()

# Filter aquifer_measurements to include only stations with instances in all years from 2010 to 2017
aquifer_measurements = aquifer_measurements[aquifer_measurements['station_id'].isin(station_ids_all_years)]

# Display the filtered DataFrame
print(aquifer_measurements)


# Filter the locations file, so it only includes the remaining stations
aquifer_stations = aquifer_stations[aquifer_stations['id'].isin(station_ids_all_years)]

In [ ]:
# Get the names of all of the aquifer stations that are in final selction
aquifer_stations.to_csv("aquifer_stations.csv", index=False)

# Print filtered station ids
print(len(station_ids_all_years))

### Weather data

In [ ]:
# Convert 'time' column from Unix timestamp to datetime
weather['time'] = pd.to_datetime(weather['time'], unit='s')

# Group by location_id and count the number of unique years
location_years_counts = weather.groupby('location_id')['time'].apply(lambda x: x.dt.year.nunique())

# Filter location_ids that have instances in all years from 2010 to 2017
location_ids_all_years = location_years_counts[location_years_counts == 8].index.tolist()

# Filter weather to include only location with instances in all years from 2010 to 2017
weather = weather[weather['location_id'].isin(location_ids_all_years)]

# Save as a .csv file
weather.to_csv("weather.csv", index=False)
print(weather)

# Fetch all of the location_id's
location_ids = weather['location_id'].unique()

print("Locations with instances all years from 2010 to 2017:")
print(len(location_ids))

# Filter the locations that are in the final locations
weather_locations = weather_locations[weather_locations['id'].isin(location_ids_all_years)]

### Missing values

In [ ]:
# How many missing values are there in the dataframes
print(weather.isna().sum().sum())
print(aquifer_measurements.isna().sum().sum())

In [ ]:
# Remove the columns with most missing values
weather = weather.drop('sun_duration', axis=1)
weather = weather.drop('snow_depth', axis=1)

print(weather)

In [ ]:
weather.to_csv("weather.csv", index=False)

In [ ]:
# Check how many rows are there for each id
row_count = weather.groupby('location_id').size()
print(row_count)

# Compare row counts with 2991
different_counts = row_count[row_count != 2922]
print("Different counts:")
print(different_counts)

In [ ]:
# Remove the column level
aquifer_measurements = aquifer_measurements.drop('level', axis=1)

In [ ]:
# Check how many rows are there for each id
row_count_aq = aquifer_measurements.groupby('station_id').size()
print(row_count_aq)

In [ ]:
# Compare row counts with 2850
over_treshold = row_count_aq[row_count_aq >= 2850]

# Keep only these id's
ids_to_keep = over_treshold.index

#Filter the aquifer_measurements
aquifer_measurements = aquifer_measurements[aquifer_measurements['station_id'].isin(ids_to_keep)]

### Fixing the date columns

In [ ]:
# Rename 'time' column to 'date'
weather = weather.rename(columns={'time': 'date'})

# Remove the hour from date
weather['date'] = weather['date'].dt.date

# Cast aquifer_measurements['date'] to date_time
aquifer_measurements['date'] = pd.to_datetime(aquifer_measurements['date'])

### Splitting the dataframes based on the stations

In [ ]:
# Make a dictionary with dataframes for specific aquifer stations
aquifer_measurements_by_stations = {aquifer: data for aquifer, data in aquifer_measurements.groupby('station_id')}

# Make a dictionary with dataframes for specific weather locations
weather_by_locations = {location: data for location, data in weather.groupby('location_id')}

### Completing the aquifer_stations dataset

In [ ]:
# Remove the parentheses and their contents from the 'location' column
aquifer_stations['name'] = aquifer_stations['name'].str.replace(r'\s*\(.*?\)', '', regex=True)

In [ ]:
# Filter the stations, so it only includes the remaining stations
aquifer_stations = aquifer_stations[aquifer_stations['id'].isin(ids_to_keep)]

In [ ]:
# Replace the pattern in the 'name' column
aquifer_stations['name'] = aquifer_stations['name'].str.replace(r'^Lj[^-]*-', 'Ljubljana ', regex=True)

In [ ]:
# Initialize geocoder with a specific user-agent
geolocator = Nominatim(user_agent="geocoder_for_slovenia")

# Function to geocode a place name with retries
def geocode_place(place, retries=3, delay=1):
    for i in range(retries):
        try:
            location = geolocator.geocode(place)
            time.sleep(delay) #nominatim supports only 1 query/second
            if location:
                return location.latitude, location.longitude
            else:
                return None, None
        except (GeocoderTimedOut, GeocoderServiceError) as e:
            print(f"Error geocoding {place}: {e}, retrying in {delay} seconds...")
            time.sleep(delay)
    return None, None
# Geocode each station name
aquifer_stations['latitude'], aquifer_stations['longitude'] = zip(*aquifer_stations['name'].apply(geocode_place))

In [ ]:
aquifer_stations.to_csv("aquifer_stations.csv", index=False)

### Plotting the aquifer stations and weather locations on a map

In [ ]:
#Initialize a map centered on Slovenia
slovenia_map = folium.Map(location=[46.151241, 14.995463], zoom_start=8)

# Add weather locations to the map (red colour)
for _, row in weather_locations.iterrows():
    folium.Marker(
        location=[row['lat'], row['lng']],
        popup=row['id'],
        icon=folium.Icon(color='red')
    ).add_to(slovenia_map)

# Add aquifer stations to the map (blue colour)
for _, row in aquifer_stations.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row['id'],
        icon=folium.Icon(color='blue')
    ).add_to(slovenia_map)


# Save the map to an HTML file
slovenia_map.save('../data/interim/slovenia_map.html')

### Combining the weather and aquifer data

In [ ]:
# Find the closest weather stations to the aquifer stations

# Initialize dictionary to store closest locations
closest_locations = {}

# Iterate through each location in aquifer_stations
for idx1, row1 in aquifer_stations.iterrows():
    closest_location = None
    min_distance = float('inf')
    
    # Iterate through each location in weather_locations
    for idx2, row2 in weather_locations.iterrows():
        # Calculate distance using geopy
        dist = distance((row1['latitude'], row1['longitude']), (row2['lat'], row2['lng'])).km
        
        if dist < min_distance:
            min_distance = dist
            closest_location = row2['id']
    
    # Store closest location_id in dictionary
    closest_locations[row1['id']] = closest_location

In [ ]:
print(closest_locations)

In [ ]:
# Merging the dictionaries


# Initialize an empty dictionary to store combined dataframes
combined_data = {}

# Iterate through mapping_dict and merge corresponding dataframes based on 'date'
for id1, loc_key in closest_locations.items():
    if id1 in aquifer_measurements_by_stations and loc_key in weather_by_locations:
        df1 = aquifer_measurements_by_stations[id1]
        df2 = weather_by_locations[loc_key]
        
        # Cast both 'date' columns to datetime
        df1['date'] = pd.to_datetime(df1['date'])
        df2['date'] = pd.to_datetime(df2['date'])

        # Merge dataframes based on 'date'
        merged_df = pd.merge(df1, df2, on='date', how='outer')
        
        # Store merged dataframe in combined_data
        combined_data[id1] = merged_df
        

### Missing values based on stations

In [ ]:
for key, data in combined_data.items():
    print(f"{key}: {data.isna().sum().sum()}")

In [ ]:
missing_values = []
for key, data in combined_data.items():
    num_missing = data.isna().sum().sum()
    if num_missing < 200:
        missing_values.append(key)
print(len(combined_data.keys()))
print(len(missing_values))
print(missing_values)


In [ ]:
print(combined_data[30015].isna().sum())

In [ ]:
# Filter the dictionary to only include keys in the list
combined_data = {key: combined_data[key] if key in combined_data else None for key in missing_values}

for key, data in combined_data.items():
    print(f"{key}: {data.shape[0]}")

### New feature generation

In [ ]:
# Shift one feature for 3 days and fill the NaNs with the first value in the column

'''first_value = combined_data[10005]['temperature_avg'].iloc[0]
# Create a new column with values shifted by 3 days ahead
combined_data[10005]['temperature_avg_shift3'] = combined_data[10005]['temperature_avg'].shift(3)
combined_data[10005]['temperature_avg_shift3'] = combined_data[10005]['temperature_avg_shift3'].fillna(first_value)
print(combined_data[10005]['temperature_avg_shift3'])
print(combined_data[10005]['temperature_avg'].iloc[0])'''

In [ ]:
# Generate a new column altitude_diff that contains the altitude difference
# between consecutive days

for key, data in combined_data.items():
    data['altitude_diff'] = data['altitude'].diff()
    data['altitude_diff'] = data['altitude_diff'].fillna(0)

In [ ]:
# Shifting the appropriate columns for 1 to 10 days ahead

# Specify the columns to shift
columns_to_shift = ['day_time', 'precipitation', 'snow_accumulation', 'temperature_avg',
       'temperature_min', 'temperature_max', 'cloud_cover_avg',
       'cloud_cover_min', 'cloud_cover_max', 'dew_point_avg', 'dew_point_min',
       'dew_point_max', 'humidity_avg', 'humidity_min', 'humidity_max',
       'pressure_avg', 'pressure_min', 'pressure_max', 'uv_index_avg',
       'uv_index_min', 'uv_index_max', 'precipitation_probability_avg',
       'precipitation_probability_min', 'precipitation_probability_max',
       'precipitation_intensity_avg', 'precipitation_intensity_min',
       'precipitation_intensity_max', 'altitude_diff']

# Iterate over all of the dataframes in the dictionary
for key, data in combined_data.items():
    # Iterate over all of the columns in the columns_to_shift
    for column in columns_to_shift:
        # Iterate over all shifts
        for shift in range (1, 16):
            first_value = data[column].iloc[0]
            data[f'{column}_shift{shift}'] = data[column].shift(shift)
            # Fill the first values (NaN) with the first values from original columns
            data[f'{column}_shift{shift}'] = data[f'{column}_shift{shift}'].fillna(first_value)

In [ ]:
# Calculating the averages for all of the features (2 to 10 days)

# Specify the columns to average
columns_to_average = ['day_time', 'precipitation', 'snow_accumulation', 'temperature_avg',
       'temperature_min', 'temperature_max', 'cloud_cover_avg',
       'cloud_cover_min', 'cloud_cover_max', 'dew_point_avg', 'dew_point_min',
       'dew_point_max', 'humidity_avg', 'humidity_min', 'humidity_max',
       'pressure_avg', 'pressure_min', 'pressure_max', 'uv_index_avg',
       'uv_index_min', 'uv_index_max', 'precipitation_probability_avg',
       'precipitation_probability_min', 'precipitation_probability_max',
       'precipitation_intensity_avg', 'precipitation_intensity_min',
       'precipitation_intensity_max', 'altitude_diff']

# Iterate over all of the dataframes in the dictionary
for key, data in combined_data.items():
    # Iterate over all of the columns in the columns_to_average
    for column in columns_to_average:
        # Iterate over all average window sizes
        for average in range (2, 16):
            data[f'{column}_average{average}'] = data[column].rolling(window=average, min_periods=1).mean()

In [ ]:
# Open the file in write mode
with open('columns.txt', 'w') as file:
    # Iterate through the list elements
    for item in combined_data[10005].columns:
        # Write each element to the file followed by a newline
        file.write(item + '\n')